In [345]:
import pandas as pd
from datetime import datetime

# Define Variables

In [350]:
path_to_dir = "/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/2024-01-01_to_2025-06-05/2025-06-05 19:48:58.031710/"
path_to_data=f"{path_to_dir}final_costs_with_azure_and_aws_and_db_2024-01-01_2025-06-05.csv"
expected_cost = {
    "azure": {"2025-05":212,"2025-04":199},
    "stackit": {"2025-05":2336.94,"2025-04":2394.9},
    "aws": {"2025-05":3940.65,"2025-04":3996.67},
    "databricks": {"2025-05":1431.79,"2025-04":1152.16}
}
path_to_save = f"data/{datetime.today().date()}"
path_to_save

'data/2025-06-05'

In [351]:
import os
os.makedirs(path_to_save, exist_ok=True)

# Load data

In [352]:
df = pd.read_csv(path_to_data)
df = df[[col for col in df.columns if "Unnamed" not in col]]
df

/var/folders/yg/6h14_gl52p7_rhsrqbdjv0br0000gn/T/ipykernel_23022/412615877.py:1: DtypeWarning: Columns (4,5,6,7,8,9,12,13,14,15,17,18,19,20,22,23,24,27,28,31,38,39,40,41,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_data)


,Deployments,tenant,deployment_node_name,audience,schedule,use_conversion_table_targeting,max_model_age_in_days,node_retraining_reset,Prefect_Deployments,timestamp,...,total_charge_of_serviceName,count_services,charge_by_service,effective_price,billing_currency,resource_id,resource_name,resource_group,amortized_cost,cluster_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.188056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.846250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.068611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50.256667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,58.632778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,USD,NaN,NaN,node_types,NaN,NaN
123446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,USD,NaN,NaN,node_types,NaN,NaN
123447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,USD,NaN,NaN,node_types,NaN,NaN
123448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,USD,NaN,NaN,node_types,NaN,NaN


In [353]:
df["date"] = pd.to_datetime(df["date"], format="mixed", utc=True).dt.strftime("%Y-%m-%d")
#df["date"] = pd.to_datetime(df["date"], utc=True)
#df["date"].unique()

# Check costs of the last april 2025 and may 2025 and compare with expected data

In [354]:
min_date = "2025-04-01"
max_date = "2025-05-31"
costs_april_may = df[(df["date"]>=min_date)&(df["date"]<=max_date)]
print(f"Daterange: {costs_april_may['date'].min()} - {costs_april_may['date'].max()} ")

Daterange: 2025-04-01 - 2025-05-31 


In [355]:
# special case databricks
databricks_resources = costs_april_may[costs_april_may["cloud"]=="databricks"]["resource_group"].unique()
# keep just one (otherwise costs will be tripeled)
databricks_resources = databricks_resources[0:len(databricks_resources)-1]
costs_april_may = costs_april_may[costs_april_may["resource_group"].isin(databricks_resources)==False]
databricks_resources

array(['cluster', 'resource'], dtype=object)

In [356]:
# special case stackit
costs_april_may = costs_april_may.reset_index(drop=True)
# stackit = costs_april_may[costs_april_may["cloud"]=="stackit"]
# duplicated_stackit_indexes = stackit[stackit.duplicated(subset=["date","serviceName","charge"])].index
# print(duplicated_stackit_indexes)
# costs_april_may = costs_april_may.drop(index=duplicated_stackit_indexes)



In [357]:
costs_april_may["month"] = pd.to_datetime(costs_april_may["date"], format="mixed")
costs_april_may["month"] = pd.to_datetime(costs_april_may["date"], format="mixed", utc=True).dt.to_period("M")
costs_april_may_grouped = costs_april_may.groupby(by=["cloud","month"])["charge"].sum()
costs_april_may_grouped = pd.DataFrame(costs_april_may_grouped).reset_index()
costs_april_may_grouped["month"] = costs_april_may_grouped["month"].astype("str")

/var/folders/yg/6h14_gl52p7_rhsrqbdjv0br0000gn/T/ipykernel_23022/791119941.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  costs_april_may["month"] = pd.to_datetime(costs_april_may["date"], format="mixed", utc=True).dt.to_period("M")


In [358]:
costs_april_may_grouped["expected_costs"] = costs_april_may_grouped[["cloud","month"]].apply(lambda x: expected_cost[x["cloud"]][x["month"]], axis=1)
costs_april_may_grouped["diff"] = costs_april_may_grouped["charge"]-costs_april_may_grouped["expected_costs"]
costs_april_may_grouped["diff_in_perc"] = (costs_april_may_grouped["charge"]-costs_april_may_grouped["expected_costs"])/costs_april_may_grouped["expected_costs"]*100
costs_april_may_grouped

,cloud,month,charge,expected_costs,diff,diff_in_perc
0,aws,2025-04,3996.671023,3996.67,0.001023,0.000026
1,aws,2025-05,3940.650573,3940.65,0.000573,0.000015
2,azure,2025-04,198.883740,199.00,-0.116260,-0.058422
3,azure,2025-05,212.338799,212.00,0.338799,0.159811
4,databricks,2025-04,1152.165437,1152.16,0.005437,0.000472
5,databricks,2025-05,1431.791805,1431.79,0.001805,0.000126
6,stackit,2025-04,2394.898026,2394.90,-0.001974,-0.000082
7,stackit,2025-05,2336.942503,2336.94,0.002503,0.000107


In [ ]:
quit

## Debugging section

In [ ]:
expected_cost["stackit"]

In [ ]:
# Error due to 0 values

In [ ]:
temp = costs_april_may[costs_april_may["cloud"]=="stackit"]
temp = temp[(temp["month"]=="2025-05")]#.drop_duplicates(subset=["date","serviceName","charge","account","duration"])#&(temp["duration"]!=0)
temp.drop_duplicates(subset=["date","serviceName","total_charge_of_serviceName"])["total_charge_of_serviceName"].sum()
temp

In [ ]:
temp0 = temp[(temp["sum_duration_serviceName"]==0)&(temp["duration"]==0)]
print(temp0.shape)
temp0["count_services"] = temp0.groupby(by=["date","serviceName"])["serviceName"].transform("count")
temp0["charge_by_service"] = temp0["charge"]/temp0["count_services"]
temp0[["date","serviceName","charge","count_services","charge_by_service"]].drop_duplicates() #TODO: Handle somehow
print(len(temp0))
temp = pd.merge(temp, temp0[["date","serviceName","charge","count_services","charge_by_service","tenant","node_name","timestamp"]], on=["date","serviceName","charge","tenant","node_name","timestamp"],how="left")
import numpy as np
temp["charge"] = np.where(temp["charge_by_service"].isnull()==False, temp["charge_by_service"], temp["charge"])

In [ ]:
temp["charge"].sum()

In [ ]:
total_charge_sum = temp.drop_duplicates(subset=["date","serviceName","total_charge_of_serviceName"]).groupby(by=["serviceName"])["total_charge_of_serviceName"].sum()
total_charge_sum

In [ ]:
charge_sum = temp.groupby(by=["serviceName"])["charge"].sum().sort_index()
charge_sum

In [ ]:
charge_merge = pd.merge(total_charge_sum, charge_sum, on="serviceName")
charge_merge["equal"] = charge_merge["total_charge_of_serviceName"].round(2) == charge_merge["charge"].round(2)
charge_merge

In [ ]:
temp[["date","charge","serviceName","cloud","total_charge_of_serviceName","part_of_costs","sum_duration_serviceName","node_name","machine.type","serviceCategoryName"]].sort_values(by=["date","serviceName"])

## Test just aws - april and may only

In [ ]:
aws = pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/2025-04-01_to_2025-04-30/2025-06-03 19:18:08.255219/aws_costs_pd_2025-04-01_to_2025-04-30.csv")
aws["charge"].sum()

In [ ]:
expected_cost["aws"]

In [ ]:
aws_may = pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/2025-05-01_to_2025-05-31/2025-06-03 19:18:08.255219/aws_costs_pd_2025-05-01_to_2025-05-31.csv")
aws_may["charge"].sum()


In [ ]:
aws_may["date"].value_counts()

# Test just azure for april and may

In [ ]:
expected_cost["azure"]

In [ ]:
azure_may = pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/2025-05-01_to_2025-05-31/2025-06-03 19:18:08.255219/azure_costs_pd_2025-05-01_2025-05-31.csv")

In [ ]:
azure_may["charge"].sum()

In [ ]:
azure_may["date"].value_counts()

In [ ]:
azure_april= pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/2025-04-01_to_2025-04-30/2025-06-03 19:18:08.255219/azure_costs_pd_2025-04-01_2025-04-30.csv")
azure_april

In [ ]:
azure_april["charge"].sum()

In [ ]:
#azure_april["date"].value_counts().sort_index()

# Test just stackit for april and may

In [ ]:
expected_cost["stackit"]

In [ ]:
stackit_april= pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/2025-04-01_to_2025-04-30/2025-06-05 18:33:21.031067/stackit_costs_merged_2025-04-01_2025-04-30.csv")
stackit_april= pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/2025-04-01_to_2025-04-30/2025-06-05 18:33:21.031067/final_costs_stackit_2025-04-01_2025-04-30.csv")

In [ ]:
stackit_april["charge"].sum()

In [ ]:
stackit_april.drop_duplicates(subset=["serviceName","date","total_charge_of_serviceName"])["total_charge_of_serviceName"].sum()

In [ ]:
stackit_may = pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/2025-05-01_to_2025-05-31/2025-06-03 19:18:08.255219/stackit_costs_merged_2025-05-01_2025-05-31.csv")

In [ ]:
stackit_may["charge"].sum()

# Test just databricks for april and may

In [ ]:
expected_cost["databricks"]

In [ ]:
db_april= pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/2025-04-01_to_2025-04-30/2025-06-03 19:18:08.255219/databricks_data_2025-04-01_2025-04-30.csv")


In [ ]:
db_april.groupby("resource_group")["charge"].sum()

In [ ]:
db_may = pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/2025-05-01_to_2025-05-31/2025-06-03 19:18:08.255219/databricks_data_2025-05-01_2025-05-31.csv")

In [ ]:
db_may.groupby("resource_group")["charge"].sum()

## Test sum up stackit costs

In [ ]:
expected_cost["stackit"]

In [ ]:
df_stackit = df[df["cloud"]=="stackit"]

In [ ]:
df_stackit[df_stackit["Prefect_Deployments"]=="targeting"]["date"].value_counts().sort_index()

In [ ]:
df_stackit_temp = df_stackit[df_stackit["date"]=="2025-05-27"]
df_stackit_temp[["Deployments","tenant","deployment_node_name","charge","node_name","duration","date","serviceName","machine.type"]].sort_values(by="serviceName")

In [ ]:
df_stackit_temp.groupby(by=["date","serviceName"])["duration"].sum()

In [ ]:
df_stackit_temp["sum_duration_serviceName"] = df_stackit_temp.groupby(by=["date","serviceName"])["duration"].transform("sum")
df_stackit_temp["part_of_costs"] = df_stackit_temp["duration"]/df_stackit_temp["sum_duration_serviceName"]
df_stackit_temp["total_charge_of_serviceName"] = df_stackit_temp["charge"]
df_stackit_temp["charge"] = df_stackit_temp["total_charge_of_serviceName"]*df_stackit_temp["part_of_costs"]
df_stackit_temp[["Deployments","tenant","deployment_node_name","total_charge_of_serviceName","charge","node_name","duration","sum_duration_serviceName","part_of_costs","date","serviceName","machine.type"]].sort_values(by="serviceName")

In [ ]:
df_stackit_temp.groupby(by=["date","serviceName"])["charge"].sum()